In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('test_data.csv')
# df = df.rename(columns={
#     "item_id": "id", 
#     "": "datetime",
#     "Mean": "mean_temp"
# })

In [4]:
values = [tuple(x) for x in df.to_numpy()]
cols = ','.join(list(df.columns))
query  = "INSERT INTO %s (%s) VALUES %%s" % ('table', cols)

In [32]:
import psycopg2

def db_connection():
    conn = psycopg2.connect(
    user='postgres',
    password='postgres',
    host='localhost',
    port='5432',
    database='postgres'
    )
    return conn

In [33]:
conn = db_connection()
cur = conn.cursor()

In [40]:
request_body = {"name":"nuno",
                "price":12.34,
                "stock":4}
vals = ','.join([f'{i}=\'{j}\'' for i,j in request_body.items()])
query = ''' DO $$
            BEGIN
                UPDATE item SET %s WHERE id=%%s;
                IF not found then RAISE EXCEPTION 'Item not found';
                END IF;
            END $$;''' % vals
# vals = ','.join([f"{i}='%s'" for i in request_body.keys()])
# query = ''' DO $$
#             BEGIN
#                 UPDATE item SET %s WHERE id=%%s;
#                 IF not found then RAISE EXCEPTION 'Item not found';
#                 END IF;
#             END $$;''' % vals
cur.execute(query,[1])
conn.commit()

In [39]:
list(request_body.values()) + [1]

['nuno', 12.34, 4, 1]

In [16]:
a = "%s"
a % 2

'2'

In [31]:
a = "%s"
q = "%s" % a
q

'%s'

In [14]:
'split f'.split()

['split', 'f']

In [11]:
'''%s,sjdfkd, sidjfsjf '%%' ''' % 5

"5,sjdfkd, sidjfsjf '%' "

In [ ]:
'''INSERT INTO cart_item (%s) VALUES %%s
                   ON CONFLICT (item_id,cart_client_id)
                   DECLARE available_stock INTEGER;
                   SELECT stock FROM item WHERE item.id = {a} INTO available_stock;
                    IF NEW.quantity > available_stock THEN
                        RAISE EXCEPTION 'Not enough stock :(.';
                    END IF;
                   DO UPDATE SET quantity = cart_item.quantity + EXCLUDED.quantity
                   WHERE cart_item.item_id = EXCLUDED.item_id AND cart_item.cart_client_id = EXCLUDED.cart_client_id;''' % cols

In [ ]:
INSERT INTO cart_item (%(cols)s) VALUES %%s
                    ON CONFLICT (item_id,cart_client_id)
                    DECLARE available_stock INTEGER;
                    SELECT stock FROM item WHERE item.id = {request_body['item_id']} INTO available_stock;
                    IF {request_body['quantity']} > available_stock THEN
                        RAISE EXCEPTION 'Not enough stock :(.';
                    END IF;
                    DO UPDATE SET quantity = cart_item.quantity + {request_body['quantity']}
                    WHERE cart_item.item_id = {request_body['item_id']} AND cart_item.cart_client_id = client123;''' % {"thisId":request_body['item_id'],
                                                                                                                        "thisQuantity":request_body['quantity'],
                                                                                                                        "cols":cols}

In [5]:
"Hello, %(name)s! You're %(name)s years old." % {"name":"Jane"}

"Hello, Jane! You're Jane years old."

In [3]:
{1,2,3,4}.issubset({1,2,3})

False

In [75]:
#category
df.loc[:,['category']].rename(columns={"category": "name"}).drop_duplicates()

,name
0,Food
1,Toys
2,Accessories


In [33]:
#manufacturer
df.loc[:,['manufacturer']].rename(columns={"manufacturer": "name"}).drop_duplicates()

,name
0,ABC Pet Foods
1,Pet Fun Co.
2,Pet Accessories Inc.
3,Healthy Pets Ltd.
4,Cat Playtime
5,Leash Master
6,Protein Paws
7,SqueakMaster
8,GroomIt
9,Organic Pets Inc.


In [84]:
pd.factorize(df['manufacturer'])[0] + 1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])

In [81]:
# #product
# df.loc[:,['item_id',
#           'name',
#           'category',
#           'price',
#           'stock',
#           'description',
#           'manufacturer',
#           'weight',
#           'image_url']].rename(columns={"item_id": "id"})

#product
product = df.loc[:,['name',
          'price',
          'stock',
          'description',
          'weight',
          'image_url']]
product['manufacturer_id'] = pd.factorize(df['manufacturer'])[0]
product['category_id'] = pd.factorize(df['category'])[0]
product

,name,price,stock,description,weight,image_url,manufacturer_id,category_id
0,Item A,9.99,1000,Premium pet food,2.0,https://example.com/item-a.jpg,0,0
1,Item B,5.99,500,Interactive toy for pets,0.5,https://example.com/item-b.jpg,1,1
2,Item C,12.99,300,Pet collar for dogs,0.2,https://example.com/item-c.jpg,2,2
3,Item D,7.49,750,Healthy cat food,1.5,https://example.com/item-d.jpg,3,0
4,Item E,3.49,800,Catnip ball for cats,0.3,https://example.com/item-e.jpg,4,1
5,Item F,8.99,450,Pet leash for dogs,0.8,https://example.com/item-f.jpg,5,2
6,Item G,6.79,600,High-protein dog food,1.2,https://example.com/item-g.jpg,6,0
7,Item H,4.29,350,Squeaky chew toy for dogs,0.6,https://example.com/item-h.jpg,7,1
8,Item I,15.49,200,Cat grooming kit,0.9,https://example.com/item-i.jpg,8,2
9,Item J,10.99,900,Organic cat food,1.8,https://example.com/item-j.jpg,9,0


In [89]:
def algo():
    assert type('d') is int, 'its not'
    return 'okay'
algo()

AssertionError: its not

In [94]:
list({'a':1,'c':3}.values())

[1, 3]

In [106]:
set((1,2)).issubset({3,5,1,5,2})

True

In [1]:
import inspect

In [99]:
",".join(map(lambda x: f"'{str(x)}'", list({'a':1,'c':3}.values())))

"'1','3'"

In [85]:
#cart
df[~df['client_id'].isnull()].loc[:,['client_id']].drop_duplicates()

,client_id
0,client123
2,client456
3,client789


In [82]:
#client
df[~df['client_id'].isnull()].loc[:,['client_id',
          'client_name',
          'client_email'
          ]].rename(columns={"client_id": "id",
                             "client_name": "name",
                             "client_email": "email"}).drop_duplicates()

,id,name,email
0,client123,John Doe,johndoe@example.com
2,client456,Jane Smith,janesmith@example.com
3,client789,Mark Johnson,markjohnson@example.com


In [67]:
df

,item_id,name,category,price,stock,description,manufacturer,weight,image_url,client_id,client_name,client_email,purchase_date,purchase_quantity
0,1,Item A,Food,9.99,1000,Premium pet food,ABC Pet Foods,2.0,https://example.com/item-a.jpg,client123,John Doe,johndoe@example.com,2023-09-01T10:30:00Z,2.0
1,2,Item B,Toys,5.99,500,Interactive toy for pets,Pet Fun Co.,0.5,https://example.com/item-b.jpg,client123,John Doe,johndoe@example.com,2023-09-03T15:45:00Z,1.0
2,3,Item C,Accessories,12.99,300,Pet collar for dogs,Pet Accessories Inc.,0.2,https://example.com/item-c.jpg,client456,Jane Smith,janesmith@example.com,2023-08-25T11:20:00Z,1.0
3,4,Item D,Food,7.49,750,Healthy cat food,Healthy Pets Ltd.,1.5,https://example.com/item-d.jpg,client789,Mark Johnson,markjohnson@example.com,2023-09-05T14:10:00Z,3.0
4,5,Item E,Toys,3.49,800,Catnip ball for cats,Cat Playtime,0.3,https://example.com/item-e.jpg,NaN,NaN,NaN,NaN,NaN
5,6,Item F,Accessories,8.99,450,Pet leash for dogs,Leash Master,0.8,https://example.com/item-f.jpg,NaN,NaN,NaN,NaN,NaN
6,7,Item G,Food,6.79,600,High-protein dog food,Protein Paws,1.2,https://example.com/item-g.jpg,NaN,NaN,NaN,NaN,NaN
7,8,Item H,Toys,4.29,350,Squeaky chew toy for dogs,SqueakMaster,0.6,https://example.com/item-h.jpg,NaN,NaN,NaN,NaN,NaN
8,9,Item I,Accessories,15.49,200,Cat grooming kit,GroomIt,0.9,https://example.com/item-i.jpg,NaN,NaN,NaN,NaN,NaN
9,10,Item J,Food,10.99,900,Organic cat food,Organic Pets Inc.,1.8,https://example.com/item-j.jpg,NaN,NaN,NaN,NaN,NaN


In [51]:
purchase = df.sort_values(by=['purchase_date'])[~df['purchase_date'].isnull()].loc[:,['purchase_date','client_id','price','purchase_quantity']]
purchase['id'] = [0,1,2,3]
purchase['price'] = purchase['price']*purchase['purchase_quantity']
purchase = purchase[['id','purchase_date','price','client_id']]
purchase

/tmp/ipykernel_30758/3375404978.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  purchase = df.sort_values(by=['purchase_date'])[~df['purchase_date'].isnull()].loc[:,['purchase_date','client_id','price','purchase_quantity']]


,id,purchase_date,price,client_id
2,0,2023-08-25T11:20:00Z,12.99,client456
0,1,2023-09-01T10:30:00Z,19.98,client123
1,2,2023-09-03T15:45:00Z,5.99,client123
3,3,2023-09-05T14:10:00Z,22.47,client789


In [54]:
purchase_item = df.sort_values(by=['purchase_date'])[~df['purchase_date'].isnull()].loc[:,['purchase_quantity','item_id']]
purchase_item['id_purchase'] = [0,1,2,3]
purchase_item = purchase_item[['purchase_quantity','id_purchase','item_id']].rename(columns={"purchase_quantity":"quantity","id_purchase": "purchase_id"})
purchase_item

/tmp/ipykernel_30758/657203008.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  purchase_item = df.sort_values(by=['purchase_date'])[~df['purchase_date'].isnull()].loc[:,['purchase_quantity','item_id']]


,quantity,purchase_id,item_id
2,1.0,0,3
0,2.0,1,1
1,1.0,2,2
3,3.0,3,4
